In [1]:
# SystemSetup equivalent in Python
# =================================
# Initialize general system parameters for LED array microscope

import numpy as np

# Fourier operators
F = lambda x: np.fft.fftshift(np.fft.fft2(x))
Ft = lambda x: np.fft.ifft2(np.fft.ifftshift(x))
row = lambda x: x.flatten()[np.newaxis, :]

In [2]:
# =====================================================================
# wavelength of illumination, assume monochromatic
# R: 624.4nm +- 50nm
# G: 518.0nm +- 50nm
# B: 476.4nm +- 50nm
# =====================================================================
lambda_m = 0.514e-6  # meters

In [3]:
# =====================================================================
# numerical aperture of the objective
# =====================================================================
NA = 0.15
um_m = NA / lambda_m  # maximum spatial frequency set by NA
dx0 = 1 / (um_m * 2)  # system resolution based on the NA

In [4]:
# =====================================================================
# magnification of the system
# =====================================================================
mag = 8.14
dpix_c = 6.5e-6  # pixel size on the sensor (meters)
dpix_m = dpix_c / mag  # effective pixel size on object plane

In [25]:
# =====================================================================
# number of pixels in image patch (single-k assumption)
# =====================================================================
# Np should be set before this script is run
# Example: Np = 256

Np = 256
FoV = Np * dpix_m  # Field of View in object space

# sampling size at Fourier plane
if Np % 2 == 1:
    du = 1 / (dpix_m * (Np - 1))
else:
    du = 1 / FoV

# generate cutoff window by NA
m = np.arange(1, Np + 1)
mm, nn = np.meshgrid(m - (Np + 1) // 2, m - (Np + 1) // 2)
ridx = np.sqrt(mm**2 + nn**2)
um_idx = um_m / du
w_NA = (ridx < um_idx).astype(np.float64)

# support of OTF is 2x ATF(NA)
Ps_otf = (ridx < 2 * um_idx).astype(np.float64)

phC = np.ones((Np, Np))
aberration = np.ones((Np, Np))
pupil = w_NA * phC * aberration

du

4891.826923076924

In [7]:
# =====================================================================
# set up image coordinates
# =====================================================================
ncent = np.array([1080, 1280])  # original image center
# nstart = np.array([981, 1181])  # user must define before running
nstart = np.array([981, 1181])
img_ncent = nstart - ncent + Np / 2
img_center = (nstart - ncent + Np / 2) * dpix_m
img_start = nstart * dpix_m
img_end = (nstart + Np) * dpix_m

In [20]:
# =====================================================================
# LED array geometries and derived quantities
# =====================================================================
ds_led = 8.125e-3  # spacing between neighboring LEDs
z_led = 145e-3  # distance from LED to object

dia_led = 1  # number of LEDs used
lit_cenv = 3
lit_cenh = 4
vled = np.arange(8) +1 - lit_cenv
hled = np.arange(8) +1 - lit_cenh

hhled, vvled = np.meshgrid(hled, vled)
rrled = np.sqrt(hhled**2 + vvled**2)
LitCoord = rrled < dia_led / 2
Nled = np.sum(LitCoord)
Litidx = np.where(LitCoord)

LitCoord

array([[False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False,  True, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False]])

In [24]:
# illumination angles
dd = np.sqrt(((-hhled * ds_led - img_center[0])**2 +
              (-vvled * ds_led - img_center[1])**2 +
              z_led**2))
sin_thetav = (-hhled * ds_led - img_center[0]) / dd
sin_thetah = (-vvled * ds_led - img_center[1]) / dd
illumination_na = np.sqrt(sin_thetav**2 + sin_thetah**2)

# spatial frequency for each LED
vled_freq = sin_thetav / lambda_m
uled_freq = sin_thetah / lambda_m
idx_u = np.round(uled_freq / du).astype(int)
idx_v = np.round(vled_freq / du).astype(int)

illumination_na_used = illumination_na[LitCoord]
NBF = np.sum(illumination_na_used < NA)

idx_u, idx_v

(array([[  44,   44,   44,   44,   44,   44,   44,   43],
        [  22,   22,   22,   22,   22,   22,   22,   22],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [ -22,  -22,  -22,  -22,  -22,  -22,  -22,  -22],
        [ -44,  -44,  -44,  -44,  -44,  -44,  -44,  -43],
        [ -65,  -66,  -66,  -66,  -66,  -66,  -65,  -64],
        [ -86,  -87,  -87,  -87,  -87,  -87,  -86,  -85],
        [-106, -107, -107, -107, -107, -107, -106, -105]]),
 array([[ 65,  44,  22,   0, -22, -44, -66, -87],
        [ 66,  44,  22,   0, -22, -44, -66, -87],
        [ 66,  44,  22,   0, -22, -44, -66, -87],
        [ 66,  44,  22,   0, -22, -44, -66, -87],
        [ 65,  44,  22,   0, -22, -44, -66, -87],
        [ 65,  44,  22,   0, -22, -44, -65, -86],
        [ 64,  43,  22,   0, -22, -43, -64, -85],
        [ 63,  43,  21,   0, -21, -43, -64, -84]]))

In [ ]:
# maximum achievable spatial frequency with synthetic aperture
um_p = np.max(illumination_na_used) / lambda_m + um_m
dx0_p = 1 / (um_p * 2)
print("synthetic NA is", um_p * lambda_m)

# original object assumptions
N_obj = int(np.round(2 * um_p / du) * 2)
N_obj = int(np.ceil(N_obj / Np) * Np)
um_obj = du * N_obj / 2
dx_obj = 1 / (um_obj * 2)

# spatial grids
xp, yp = np.meshgrid(np.arange(-Np/2, Np/2) * dpix_m,
                     np.arange(-Np/2, Np/2) * dpix_m)
x0 = np.arange(-N_obj/2, N_obj/2/2) * dx_obj
xx0, yy0 = np.meshgrid(x0, x0)

# define propagation transfer function
u = np.linspace(-um_obj, um_obj - du, N_obj)
v = np.linspace(-um_obj, um_obj - du, N_obj)
u, v = np.meshgrid(u, v)

# Fresnel approximation (object defocus distance)
z0 = 0
H0 = np.exp(1j * 2 * np.pi / lambda_m * z0) * \
     np.exp(-1j * np.pi * lambda_m * z0 * (u**2 + v**2))

# Angular spectrum (optional alternative)
# dz = some_value
# H0 = np.exp(1j * 2 * np.pi * np.sqrt((1 / lambda_m**2 - u**2 - v**2) *
#              (np.sqrt(u**2 + v**2) < 1 / lambda_m)) * dz)

synthetic NA is 0.6188740081628213
